<a href="https://colab.research.google.com/github/dsavla/study/blob/main/PGMAssign2_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install libpgm
from scipy.stats import chi2_contingency
import pandas as pd
from libpgm.pgmlearner import PGMLearner
import matplotlib.pyplot as plt
import networkx as nx

  Created wheel for libpgm: filename=libpgm-1.3-cp27-none-any.whl size=55716 sha256=a2e57831500d0557156cbb6056642248f5eb707d3d21c480d600e5124b1a14db
  Stored in directory: /root/.cache/pip/wheels/bc/9c/3a/1274b1b5f544ed6bbbf3f34d3f3550df6403b627a98e4aa11c
Successfully built libpgm


In [5]:
!pip install kaggle

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{u'kaggle.json': '{"username":"dannysavla","key":"8ac2dd73ee48a1bf50013efb735c1d96"}'}

In [7]:
! mkdir ~/.kaggle

In [8]:
! cp kaggle.json ~/.kaggle/

In [9]:
! chmod 600 ~/.kaggle/kaggle.json

In [11]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              235KB  2021-09-15 20:16:36          12316  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           7457  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           3051  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-07-03 18:37:20           2913  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Jo

In [14]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 22.7MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 7.18MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 30.4MB/s]


In [13]:
!unzip test-file.zip

Archive:  test-file.zip
  inflating: tested.csv              


In [16]:
df_input = pd.read_csv('train.csv', sep=',')

In [17]:
print (df_input.keys())

Index([u'PassengerId', u'Survived', u'Pclass', u'Name', u'Sex', u'Age',
       u'SibSp', u'Parch', u'Ticket', u'Fare', u'Cabin', u'Embarked'],
      dtype='object')


In [32]:
revisedcolumns = []
for key in df_input.keys() :
    value = str(key)
    if (value not in [ 'PassengerId', 'Cabin' , 'Ticket', 'Name', 'Pclass']) :
        revisedcolumns.append(value)

print (revisedcolumns)


['Survived', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']


In [33]:
defaultlength = len(revisedcolumns)
for i in range(defaultlength):
    print(revisedcolumns[i])


Survived
Sex
Age
SibSp
Parch
Fare
Embarked


In [34]:
df_train =  df_input.dropna()
df_train = df_input[revisedcolumns]

df_train.loc[:,'Sex']  = df_train.Sex.map({'female':0 , 'male':1})
df_train.loc[:,'Embarked']  = df_train.Embarked.map({'S':0 , 'C':1, 'Q':2})
df_train.loc[:,'Fare'] = pd.cut(df_train.Fare, [df_input.Fare.min(), df_input.Fare.mean(), df_input.Fare.max()], labels =[0,1])
df_train =  df_train.dropna()

In [35]:
def check_depedency (df, srcColumn, targetColumn ) :

    df_src_targer = df[[srcColumn, targetColumn]]

    #print (df_src_targer.head())

    contigency= pd.crosstab(df_src_targer[srcColumn], df_src_targer[targetColumn])
    #print (contigency)
    c, p, dof, expected = chi2_contingency(contigency)
    alpha = 0.05
    if p <= alpha:
        #print(srcColumn , '--', targetColumn, '::', 'Dependent (reject H0)')
        print ( p, ":", srcColumn , ":" ,targetColumn )
        return 2
    else:
        #print(srcColumn , '--', targetColumn, '::', 'Independent (H0 holds true)')
        return 1

In [36]:
src = 'Survived'
columns = [src]

for y in range(defaultlength):
        target = revisedcolumns[y]
        if (src != target ) :
            reference = check_depedency(df_train, src, target)
            if  ( reference == 2) :
                columns.append(target)
print (columns)

(1.566763733592489e-45, ':', 'Survived', ':', 'Sex')
(0.000545194781522789, ':', 'Survived', ':', 'SibSp')
(9.239675790569918e-05, ':', 'Survived', ':', 'Parch')
(1.0411362374482196e-10, ':', 'Survived', ':', 'Fare')
(1.1929282478783583e-06, ':', 'Survived', ':', 'Embarked')
['Survived', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']


In [30]:
df_train_revsied = df_train[revisedcolumns]


In [31]:
print (df_train_revsied.keys())

Index([u'Survived', u'Pclass', u'Sex', u'Age', u'SibSp', u'Parch', u'Fare',
       u'Embarked'],
      dtype='object')
